In [92]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/contextualize_best_hits")

library(dplyr, verbose = FALSE)
library(tidyr)
library(scales)
library(ggrepel)
library(ggplot2)
library(data.table)
library(RColorBrewer)

myo_meta <- read.csv("../../../data/myotis_meta.csv")

In [ ]:
# ## Plot summary of peaks in reciprocal regions:

# for (i in seq_along(myo_meta$Abbr)) {
  
#   for (j in seq_along(myo_meta$Abbr)) {

#     if (i > j) {

#       # pdf(file = paste0("results/myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j]_best_hits_peak_summary.pdf"), width = 7, height = 6)

#       peaks1 <- fread(
#         file.path("resources/genes/peaks", 
#         list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], ".*_peaks.csv"))),
#         data.table = FALSE
#         )
#       peaks2 <- fread(
#         file.path("resources/genes/peaks", 
#         list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], ".*_peaks.csv"))),
#         data.table = FALSE
#         )

#       peaks1$Reciprocal_Seq <- with(peaks1, paste(Chr, Reciprocal_Start, Reciprocal_End))
#       peaks2$Reciprocal_Seq <- with(peaks2, paste(Chr, Reciprocal_Start, Reciprocal_End))
#       peaks1$Reciprocal_Length <- as.numeric(with(peaks1, Reciprocal_End - Reciprocal_Start))
#       peaks2$Reciprocal_Length <- as.numeric(with(peaks2, Reciprocal_End - Reciprocal_Start))
#       peaks1$Peak_Length <- as.numeric(with(peaks1, paste(Peak_End - Peak_Start)))
#       peaks2$Peak_Length <- as.numeric(with(peaks2, paste(Peak_End - Peak_Start)))

#       peaks1_df <- peaks1 %>%
#         dplyr::group_by(Chr, Reciprocal_Seq) %>% 
#         dplyr::summarise(
#           No.Peaks = n(),
#           No.Peaks_Rel = n()/unique(Reciprocal_Length)
#         )

#       peaks2_df <- peaks2 %>%
#         dplyr::group_by(Chr, Reciprocal_Seq) %>% 
#         dplyr::summarise(
#           No.Peaks = n(),
#           No.Peaks_Rel = n()/unique(Reciprocal_Length)
#         )

#       ## Plot distro of peak lengths:

#       print(
#         ggplot(peaks1, aes(x = Peak_Length)) + 
#           geom_histogram(bins = 100) +
#           theme_minimal() +
#           theme(axis.title.x = element_text(margin = margin(t = 15)),
#                 axis.title.y = element_blank()) +
#           labs(title = paste(myo_meta$Abbr[i], "peaks in reciprocal regions with", myo_meta$Abbr[j])) +
#           xlab("Peak length (nt)") +
#           scale_x_continuous(labels = comma) +
#           scale_y_continuous(labels = comma)
#       )

#       print(
#         ggplot(peaks2, aes(x = Peak_Length)) + 
#           geom_histogram(bins = 100) +
#           theme_minimal() +
#           theme(axis.title.x = element_text(margin = margin(t = 15)),
#                 axis.title.y = element_blank()) +
#           labs(title = paste(myo_meta$Abbr[j], "peaks in reciprocal regions with", myo_meta$Abbr[i])) +
#           xlab("Peak length (nt)") +
#           scale_x_continuous(labels = comma) +
#           scale_y_continuous(labels = comma)
#       )

#       ## Plot distro of # of peaks per reciprocal region:

#       print(
#         ggplot(peaks1_df, aes(x = No.Peaks)) + 
#           geom_histogram(bins = 50) +
#           theme_minimal() +
#           theme(axis.title.x = element_text(margin = margin(t = 15)),
#                 axis.title.y = element_blank()) +
#           labs(title = paste(myo_meta$Abbr[i], "peaks in reciprocal regions with", myo_meta$Abbr[j])) +
#           xlab("# peaks per region") +
#           scale_x_continuous(labels = comma)
#       )

#       print(
#         ggplot(peaks2_df, aes(x = No.Peaks)) + 
#           geom_histogram(bins = 50) +
#           theme_minimal() +
#           theme(axis.title.x = element_text(margin = margin(t = 15)),
#                 axis.title.y = element_blank()) +
#           labs(title = paste(myo_meta$Abbr[j], "peaks in reciprocal regions with", myo_meta$Abbr[i])) +
#           xlab("# peaks per region") +
#           scale_x_continuous(labels = comma)
#       )

#       # print(
#       #   ggplot(peaks1_df, aes(No.Peaks_Rel)) +
#       #     geom_histogram(bins = 30) +
#       #     theme_minimal() +
#       #     theme(axis.title.x = element_text(margin = margin(t = 15)),
#       #           axis.title.y = element_blank()) +
#       #     labs(title = paste(myo_meta$Abbr[i], "peaks in reciprocal regions with", myo_meta$Abbr[j])) +
#       #     xlab("# peaks per nucleotide in reciprocal regions") +
#       #     scale_x_continuous(breaks = seq(0, 1, by = .00005))
#       # )

#       # dev.off()

#     }
#   }
# }

In [93]:
## How do peaks in orthologous genes compare?

for (i in seq_along(myo_meta$Abbr)) {

  print(myo_meta$Field_Name[i])
  
  inds <- seq_along(myo_meta$Abbr)

  spec2_data <- suppressMessages(lapply(inds[inds != i], function(j) {

    ## Load in peak data for species 1: 
    ## peaks in reciprocal regions that intersect orthologous genes
    peaks1_genes <- fread(
      file.path("resources/genes/peaks", 
      list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], ".*_only.csv"))),
      data.table = FALSE
    )
    peaks1_genes <- peaks1_genes %>%
    dplyr::group_by(myotis_ali_fn) %>%
    dplyr::slice_max(as.numeric(Intersect_Start)) %>%
    dplyr::select(myotis_ali_fn, Peak_ID)

    ## Load in peak data for species 1: 
    ## Species 1 orthologous genes in reciprocal best hit regions:
    hits1_genes <- fread(
      file.path("resources/genes", 
      list.files(path = "resources/genes", pattern = paste0(myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], ".*_only.csv"))),
      data.table = FALSE
    )
    hits1_genes <- hits1_genes %>%
      dplyr::mutate(Reciprocal_Seq = paste(Chr, Reciprocal_Start, Reciprocal_End)) %>%
      dplyr::group_by(Chr, myotis_ali_fn, Reciprocal_Seq) %>%
      dplyr::summarise(human_gene_name = unique(human_gene_name))

    ## Load in peak data for species 2: 
    ## peaks in reciprocal regions that intersect orthologous genes
    peaks2_genes <- fread(
      file.path("resources/genes/peaks", 
      list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], ".*_only.csv"))),
      data.table = FALSE
    )
    peaks2_genes <- peaks2_genes %>%
      dplyr::group_by(myotis_ali_fn) %>%
      dplyr::slice_max(as.numeric(Intersect_Start)) %>%
      dplyr::select(Peak_ID)

    ## Load in peak data for species 2: 
    ## Species 1 orthologous genes in reciprocal best hit regions:  
    hits2_genes <- fread(
      file.path("resources/genes", 
      list.files(path = "resources/genes", pattern = paste0(myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], ".*_only.csv"))),
      data.table = FALSE
    )
    hits2_genes <- hits2_genes %>%
      dplyr::mutate(Reciprocal_Seq = paste(Chr, Reciprocal_Start, Reciprocal_End)) %>%
      dplyr::group_by(Chr, myotis_ali_fn, Reciprocal_Seq) %>%
      dplyr::summarise(human_gene_name = unique(human_gene_name))

    ## Genes not present in peak files implies no peak at that location:
    hits1_status <- merge(hits1_genes, peaks1_genes, by = "myotis_ali_fn", all.x = TRUE)
    hits1_status$Peak <- TRUE
    hits1_status$Peak[is.na(hits1_status$Peak_ID)] <- FALSE
    hits2_status <- merge(hits2_genes, peaks2_genes, by = "myotis_ali_fn", all.x = TRUE)
    hits2_status$Peak <- TRUE
    hits2_status$Peak[is.na(hits2_status$Peak_ID)] <- FALSE

    ## Subset to orthologous genes present in both species:
    hits1_status <- hits1_status[hits1_status$myotis_ali_fn %in% hits2_status$myotis_ali_fn,]
    hits2_status <- hits2_status[match(hits1_status$myotis_ali_fn, hits2_status$myotis_ali_fn),]
    if (!identical(hits2_status$myotis_ali_fn, hits1_status$myotis_ali_fn)) {
      print(paste(i, j))
      stop("Genes do not match between species")
    }

    ## Correlation between peak status in orthologous genes:
    peak_cor <- cor(hits1_status$Peak, hits2_status$Peak)
    
    ## Of the orthologous genes in species 1, what fraction of them have differential peaks to species 2?
    is_diff <- hits1_status$Peak != hits2_status$Peak
    n_shared_peaks <- as.matrix(table(!is_diff))
    
    return(
      data.frame(
        Species = myo_meta$Field_Name[j],
        Cor = peak_cor,
        Is_Peak_Shared = rownames(n_shared_peaks),
        No.Peaks = as.numeric(n_shared_peaks[,1]), 
        Frac_Peaks = as.numeric(n_shared_peaks[,1])/length(is_diff)
      )
    ) 
    
  }))
  
  df <- do.call(rbind, spec2_data)
  df$Species <- gsub("Myotis-", "", df$Species)
  
  spec_order <- df %>%
    dplyr::filter(Is_Peak_Shared == FALSE) %>%
    dplyr::arrange(desc(Frac_Peaks))  

  df$Species <- factor(df$Species, levels = spec_order$Species)
  df$Is_Peak_Shared <- factor(df$Is_Peak_Shared, levels = c(TRUE, FALSE))

  pdf(file = paste0("results/", myo_meta$Abbr[i], "_orthologous_gene_differential_peaks.pdf"), width = 7, height = 7)

  print(
    ggplot(df, aes(x = Species, y = Frac_Peaks, fill = Is_Peak_Shared)) +
    geom_bar(stat = "identity", color = "black") +
    theme_minimal() +
    theme(plot.subtitle = element_text(margin = margin(b = 10)), 
          axis.title.x = element_blank(),
          axis.text.x = element_text(margin = margin(t = -10, b = 10), size = 11, angle = 45, hjust = 1),
          axis.title.y = element_text(margin = margin(r = 15)),
          axis.text.y = element_text(size = 11),
          axis.ticks.y = element_line(color = "black"),
          legend.position = "bottom",
          legend.direction = "horizontal",
          panel.grid = element_blank(),
          plot.margin = unit(c(1, 1, 1, 1), "cm")) +
    labs(title = myo_meta$Field_Name[i], subtitle = "Peak status in orthologous genes") +
    ylab(paste("Fraction of genes\n(present in both species)")) +
    scale_fill_manual(name = "Is peak conserved between species?", values = brewer.pal(3, "Paired"))
  ) 

  df <- df[df$Is_Peak_Shared == FALSE,]

  print(
    ggplot(df, aes(x = Frac_Peaks, y = Cor)) +
    geom_point() +
    geom_text_repel(aes(label = Species), size = 4) +
    theme_minimal() +
    theme(axis.title.x = element_text(size = 12, margin = margin(t = 15)),
          axis.text.x = element_text(size = 11),
          axis.title.y = element_text(size = 12, margin = margin(r = 15)),
          axis.text.y = element_text(size = 11),
          axis.ticks.y = element_line(color = "black"),
          legend.position = "bottom",
          legend.direction = "horizontal",
          plot.margin = unit(c(1, 1, 1, 1), "cm")) +
    labs(title = myo_meta$Field_Name[i], subtitle = "Peak status in orthologous genes") +
    xlab("Fraction of genes with differential peak status") +
    ylab("Correlation between peaks status")
  )

  dev.off()

}

[1] "Myotis-Auriculus"
[1] "Myotis-Californicus"
[1] "Myotis-Evotis"
[1] "Myotis-Lucifugus"
[1] "Myotis-Occultus"
[1] "Myotis-Thysanodes"
[1] "Myotis-Velifer"
[1] "Myotis-Volans"
[1] "Myotis-Yumanensis"


In [95]:
## How do the genes with differential peaks compare?

for (i in seq_along(myo_meta$Abbr)) {

  print(myo_meta$Field_Name[i])
  
  inds <- seq_along(myo_meta$Abbr)

  spec2_data <- suppressMessages(lapply(inds[inds != i], function(j) {

    ## Load in peak data for species 1: 
    ## peaks in reciprocal regions that intersect orthologous genes
    peaks1_genes <- fread(
      file.path("resources/genes/peaks", 
      list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], ".*_only.csv"))),
      data.table = FALSE
    )
    peaks1_genes <- peaks1_genes %>%
    dplyr::group_by(myotis_ali_fn) %>%
    dplyr::slice_max(as.numeric(Intersect_Start)) %>%
    dplyr::select(myotis_ali_fn, Peak_ID)

    ## Load in peak data for species 1: 
    ## Species 1 orthologous genes in reciprocal best hit regions:
    hits1_genes <- fread(
      file.path("resources/genes", 
      list.files(path = "resources/genes", pattern = paste0(myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], ".*_only.csv"))),
      data.table = FALSE
    )
    hits1_genes <- hits1_genes %>%
      dplyr::mutate(Reciprocal_Seq = paste(Chr, Reciprocal_Start, Reciprocal_End)) %>%
      dplyr::group_by(Chr, myotis_ali_fn, Reciprocal_Seq) %>%
      dplyr::summarise(human_gene_name = unique(human_gene_name))

    ## Load in peak data for species 2: 
    ## peaks in reciprocal regions that intersect orthologous genes
    peaks2_genes <- fread(
      file.path("resources/genes/peaks", 
      list.files(path = "resources/genes/peaks", pattern = paste0(myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], ".*_only.csv"))),
      data.table = FALSE
    )
    peaks2_genes <- peaks2_genes %>%
      dplyr::group_by(myotis_ali_fn) %>%
      dplyr::slice_max(as.numeric(Intersect_Start)) %>%
      dplyr::select(Peak_ID)

    ## Load in peak data for species 2: 
    ## Species 1 orthologous genes in reciprocal best hit regions:  
    hits2_genes <- fread(
      file.path("resources/genes", 
      list.files(path = "resources/genes", pattern = paste0(myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], ".*_only.csv"))),
      data.table = FALSE
    )
    hits2_genes <- hits2_genes %>%
      dplyr::mutate(Reciprocal_Seq = paste(Chr, Reciprocal_Start, Reciprocal_End)) %>%
      dplyr::group_by(Chr, myotis_ali_fn, Reciprocal_Seq) %>%
      dplyr::summarise(human_gene_name = unique(human_gene_name))

    ## Genes not present in peak files implies no peak at that location:
    hits1_status <- merge(hits1_genes, peaks1_genes, by = "myotis_ali_fn", all.x = TRUE)
    hits1_status$Peak <- TRUE
    hits1_status$Peak[is.na(hits1_status$Peak_ID)] <- FALSE
    hits2_status <- merge(hits2_genes, peaks2_genes, by = "myotis_ali_fn", all.x = TRUE)
    hits2_status$Peak <- TRUE
    hits2_status$Peak[is.na(hits2_status$Peak_ID)] <- FALSE

    ## Subset to orthologous genes present in both species:
    hits1_status <- hits1_status[hits1_status$myotis_ali_fn %in% hits2_status$myotis_ali_fn,]
    hits2_status <- hits2_status[match(hits1_status$myotis_ali_fn, hits2_status$myotis_ali_fn),]
    if (!identical(hits2_status$myotis_ali_fn, hits1_status$myotis_ali_fn)) {
      print(paste(i, j))
      stop("Genes do not match between species")
    }

    ## Correlation between peak status in orthologous genes:
    peak_cor <- cor(hits1_status$Peak, hits2_status$Peak)
    
    ## Of the orthologous genes in species 1, what fraction of them have differential peaks to species 2?
    is_diff <- hits1_status$Peak != hits2_status$Peak
    n_shared_peaks <- as.matrix(table(!is_diff))
    
    return(
      data.frame(
        Species = myo_meta$Field_Name[j],
        Cor = peak_cor,
        Is_Peak_Shared = rownames(n_shared_peaks),
        No.Peaks = as.numeric(n_shared_peaks[,1]), 
        Frac_Peaks = as.numeric(n_shared_peaks[,1])/length(is_diff)
      )
    ) 
    
  }))
  
  df <- do.call(rbind, spec2_data)
  df$Species <- gsub("Myotis-", "", df$Species)
  df$Is_Peak_Shared <- factor(df$Is_Peak_Shared, levels = c(TRUE, FALSE))

  pdf(file = paste0("results/", myo_meta$Abbr[i], "_orthologous_gene_differential_peaks.pdf"), width = 7, height = 7)

  spec_order <- df %>%
    dplyr::filter(Is_Peak_Shared == FALSE) %>%
    dplyr::arrange(desc(Frac_Peaks))  

  df$Species <- factor(df$Species, levels = spec_order$Species)

  print(
    ggplot(df, aes(x = Species, y = Frac_Peaks, fill = Is_Peak_Shared)) +
    geom_bar(stat = "identity", color = "black") +
    theme_minimal() +
    theme(plot.subtitle = element_text(margin = margin(b = 10)), 
          axis.title.x = element_blank(),
          axis.text.x = element_text(margin = margin(t = -10, b = 10), size = 11, angle = 45, hjust = 1),
          axis.title.y = element_text(margin = margin(r = 15)),
          axis.text.y = element_text(size = 11),
          axis.ticks.y = element_line(color = "black"),
          legend.position = "bottom",
          legend.direction = "horizontal",
          panel.grid = element_blank(),
          plot.margin = unit(c(1, 1, 1, 1), "cm")) +
    labs(title = myo_meta$Field_Name[i], subtitle = "Peak status in orthologous genes") +
    ylab(paste("Fraction of genes\n(present in both species)")) +
    scale_fill_manual(name = "Is peak conserved between species?", values = brewer.pal(3, "Paired"))
  )
  
  spec_order <- df %>%
    dplyr::filter(Is_Peak_Shared == FALSE) %>%
    dplyr::arrange(desc(No.Peaks))  

  df$Species <- factor(df$Species, levels = spec_order$Species)

  print(
    ggplot(df, aes(x = Species, y = No.Peaks, fill = Is_Peak_Shared)) +
    geom_bar(stat = "identity", position = "dodge", color = "black") +
    theme_minimal() +
    theme(plot.subtitle = element_text(margin = margin(b = 10)), 
          axis.title.x = element_blank(),
          axis.text.x = element_text(margin = margin(t = -10, b = 10), size = 11, angle = 45, hjust = 1),
          axis.title.y = element_text(margin = margin(r = 15)),
          axis.text.y = element_text(size = 11),
          axis.ticks.y = element_line(color = "black"),
          legend.position = "bottom",
          legend.direction = "horizontal",
          panel.grid = element_blank(),
          plot.margin = unit(c(1, 1, 1, 1), "cm")) +
    labs(title = myo_meta$Field_Name[i], subtitle = "Peak status in orthologous genes") +
    ylab(paste("Fraction of genes\n(present in both species)")) +
    scale_y_continuous(labels = comma) +
    scale_fill_manual(name = "Is peak conserved between species?", values = brewer.pal(3, "Paired"))
  )  

  df <- df[df$Is_Peak_Shared == FALSE,]

  print(
    ggplot(df, aes(x = Frac_Peaks, y = Cor)) +
    geom_point() +
    geom_text_repel(aes(label = Species), size = 4) +
    theme_minimal() +
    theme(axis.title.x = element_text(size = 12, margin = margin(t = 15)),
          axis.text.x = element_text(size = 11),
          axis.title.y = element_text(size = 12, margin = margin(r = 15)),
          axis.text.y = element_text(size = 11),
          axis.ticks.y = element_line(color = "black"),
          legend.position = "bottom",
          legend.direction = "horizontal",
          plot.margin = unit(c(1, 1, 1, 1), "cm")) +
    labs(title = myo_meta$Field_Name[i], subtitle = "Peak status in orthologous genes") +
    xlab("Fraction of genes with differential peak status") +
    ylab("Correlation between peaks status")
  )

  dev.off()

}

[1] "Myotis-Auriculus"
[1] "Myotis-Californicus"
[1] "Myotis-Evotis"
[1] "Myotis-Lucifugus"


In [94]:
head(df)

,Species,Cor,Is_Peak_Shared,No.Peaks,Frac_Peaks
,<fct>,<dbl>,<fct>,<dbl>,<dbl>
1,Auriculus,0.4482415,FALSE,750,0.1631499
3,Californicus,0.5845749,FALSE,243,0.1110096
5,Evotis,0.3926914,FALSE,831,0.1777160
7,Lucifugus,0.4150351,FALSE,648,0.1356216
9,Occultus,0.4427453,FALSE,667,0.1626036
11,Thysanodes,0.3047411,FALSE,816,0.1999510
